# Small Guide for Working on Alvis
## Author: Mercedes Dalman
## Date: 29/10 - 2024

### Purpose: 
This guide will walk you through connecting to the Alvis server, launching interactive sessions, setting up environments, and running both interactive and non-interactive sessions on Alvis (using my personal account).

The person following this guide is expected to have reasonable familiarity with working and connecting to a terminal. 

---

### Resources Available:
On alvis, I have a small compute - NAISS2024-22-698 -  and a small storage - NAISS2024-23-619 - project. The small compute has available 250 GPU-hours/month and the small storage has 2000 GB space. 

To see how many GPU-hours you have used, use command:

In [ ]:
projinfo

For the storage available, move to folder ~/mercedes_storage (a symlink to the small storage) and use command:

In [ ]:
du -sh

---

# 1. Logging in to Alvis on a linux-based system:

To log in to Alvis, use the following credentials:









In [ ]:
Host: alvis1.c3se.chalmers.se
Username: alinned
Password: Q>h4=H&BxF

I would suggest that you generate a ssh-key to not have to input
password each time you login:

In terminal, put: 

***

## 1.2 Setting Up SSH Key for Passwordless Login

**1. Generate SSH Key** (if not already created):

In [ ]:
ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa_alvis

**NOTE:** Leave the password prompt blank to enable passwordless access.

**2. Copy SSH Key to Alvis:**

In [ ]:
ssh-copy-id -i ~/.ssh/id_rsa_alvis.pub alinned@alvis1.c3se.chalmers.se



Enter the password provided above when prompted.

**3. Log in:** 




In [ ]:
ssh alvis1.c3se.chalmers.se

After setting up, simply type ssh al... and let autocomplete finish the hostname.

---

# 2. Launching an Interactive Session on Alvis:

---

## 2.1 Using Pre-Made cell2location_env:

To work with cell2location, a pre-configured environment is available at:
/mimer/NOBACKUP/groups/naiss2024-23-619


## **Steps to Activate Environment:**

**1. Load Required Modules:**

In [ ]:
ml purge
ml load PyTorch/2.1.2-foss-2023a-CUDA-12.1.1
ml load Python/3.11.3-GCCcore-12.3.0
ml load SciPy-bundle/2023.07-gfbf-2023a
ml load scikit-learn/1.3.1-gfbf-2023a
ml load matplotlib/3.7.2-gfbf-2023a
ml load h5py/3.9.0-foss-2023a
ml load IPython/8.14.0-GCCcore-12.3.0
ml load JupyterLab/4.0.5-GCCcore-12.3.0
ml load Pillow/10.0.0-GCCcore-12.3.0
ml load plotly.py/5.16.0-GCCcore-12.3.0

**Note:** Ensure consistent versions across modules, e.g. the GCCcore is the same (e.g. 12.3.0). 

If you want to update or change the GCCcore and 
is unsure which versions are supported on alvis, you can run the commmand:


In [ ]:
ml spider <module name (e.g. PyTorch)>

**2. Activate Virtual Environment:**

In [ ]:
CUSTOM_VENV_PATH=/cephyr/users/alinned/Alvis/mercedes_storage/cell2location_env
source $CUSTOM_VENV_PATH/bin/activate

**3. Install Additional Packages:**

In [ ]:
pip install --no-cache-dir --no-build-isolation <package_name>

*Tip: Prefer loading packages through modules for speed and compatibility. Check availability with ml spider <package_name>.*

**4. Deactivate Environment:**

In [ ]:
deactivate

---

## 2.2 Running an Interactive Session with Slurm (cell2location_env)

Alvis uses Slurm for resource allocation. For a Jupyter Notebook interactive session, use the ~/launch_jn_interactive_cell2location.sh script. 

**To run an interactive jupyter notebook with the cell2location_env, you use the command sbathc, as follows:**






In [ ]:
sbatch launch_jn_transformers_cell2location.sh

Before doing this, update the Slurm configuration after your needs.

Here’s a sample of the Slurm configuration:

In [ ]:
#!/bin/env bash
#SBATCH -A NAISS2024-22-698         # Project (use "projinfo" to list available projects)
#SBATCH -p alvis                    # Partition
#SBATCH -t 0-01:00:00               # Time (days-hours:minutes:seconds)
#SBATCH --gpus-per-node=A100:1      # GPUs and type
#SBATCH -J JOB_NAME                 # Job name
#SBATCH -o output_file.out          # Output file
#SBATCH --mail-type=ALL             # Notifications
#SBATCH --mail-user=mercedes.dalman@gu.se


*Adjust settings as needed, e.g., time, GPU type, email.*
 *To see the resources available,*
go to: https://www.c3se.chalmers.se/about/Alvis/

In the file, the virtual environment cell2location_env will then be activated. 

To run a jupyter notebook, it is important that the module JupyterNotebook is loaded (use ml spider jupyter) to find correct version.
In this case, a jypyter notebook will then be launched.

**Launch Jupyter Notebook**

In [ ]:
jupyter notebook --no-browser --ip 0.0.0.0

*The output will appear in the output_file.out once resources are allocated.*

**Note:** 
If you do not see the output_file.out immediately after running sbatch launch... then that means that you are in queue. 

This you can verify using:


In [ ]:

squeue -u alinned


If you want to cancel the session, you either use:

In [ ]:

scancel -u alinned

or if you want to not cancel all jobs but a specific one:

In [ ]:

scancel jobid 

jobid you can see using squeue -u alinned.

---

## 2.3 Accessing Jupyter Notebook in Visual Studio Code

**Connect to Host:**

In Visual Studio Code, click the small blue icon (lower-left corner) and select "Connect to Host...."

Add ssh alinned@alvis1.c3se.chalmers.se if not already configured.

**Open Jupyter Notebook:**

Go to "File" > "New File..." > "Jupyter Notebook"

**Connect to Existing Slurm Session:**

Select “Existing Jupyter Session” and paste the URL from output_file.out (e.g., http://alvis4-14:8888/tree?token=...).

(**Note:** You need to select the URL with alvis in the name (eg alvis4))

To use the cell2location_env, you then select the kernel "cell2location_env".

---

# 3. Creating a Custom Virtual Environment
To create a new environment, edit the ~/install_environment.sh script:


**1. Load Required Modules:** 

In [ ]:
ml purge
ml load PyTorch/2.1.2-foss-2023a-CUDA-12.1.1
ml load Python/3.11.3-GCCcore-12.3.0
# (repeat for other modules)

Ensure consistent versions across modules, e.g. the GCCcore is the same (e.g. 12.3.0). (See "Steps to Activate Environmnet" above.)


**2.Edit Environment Path:**

- Update CUSTOM_VENV_PATH to a new name. 

**Note:** Install the package in the small storage folder as I did with cell2location_env - there is not a lot of space in home.

**3. Uncomment Creation Line:**

In [ ]:
virtualenv --system-site-packages $CUSTOM_VENV_PATH

**4. Create Environment Using the Bash Script install_environment.sh:**

In [ ]:
./install_environment.sh

**5. Install Packages:** *See "Steps to Activate Environmnet" above.*

**6. Register Environment as Jupyter Kernel:**

In [ ]:
python -m ipykernel install --user --name=MY_ENV_NAME --display-name="MY_KERNEL_NAME"

After these steps, you can launch an interactive jupyter session in the same manner as described for cell2location above. 

**Note:** Please make a copy of the launch_jn_.. script for your own environmnent.

---

# 4. Running Non-Interactive Sessions
For non-interactive sessions, modify your launch_jn_... script to:

**1. Comment out the Jupyter Notebook line:**

In [ ]:
# jupyter notebook --no-browser --ip 0.0.0.0

**2. Uncomment the line to run a notebook directly:**

In [ ]:
ipython -c "%run SOME_NOTEBOOK.ipynb"

**3. Alternatively, Convert Notebook to Python Script:**

In [ ]:
jupyter nbconvert --to python SOME_NOTEBOOK.ipynb && python SOME_NOTEBOOK.py

**Note:** When doing this, please be aware of and modify the SBATCH configurations before running scripts. 